# Messing with the dipole module

[...]

### What am I trying to do
[...] <b>dipole module of LiteBIRD simulation</b> (<a href="https://github.com/AdriJD/beamconv">[...]</a>) 

In [61]:
import numpy as np
from matplotlib import pyplot as plt

from astropy.time import Time

# importing a slightly modified version of the spacecraft.py file
from spacecraft import SpacecraftOrbit
from spacecraft import spacecraft_pos_and_vel

# setting up the scanning strategy parameters
ctime0 = 1510000000            # initial time
mlen = 1 * 24 * 60 * 60        # mission length in seconds (one day!)

ctime0 = Time(ctime0, format="unix") #conversion to astropy object

orbit = SpacecraftOrbit(ctime0)

pos_vel = spacecraft_pos_and_vel(
            orbit,
            start_time=ctime0,
            time_span_s=mlen,
            delta_time_s=3600.
)

pos_test = pos_vel.positions_km
vel_test = pos_vel.velocities_km_s

#print(pos_test)
#print(vel_test)

In [57]:
from astropy.constants import c as c_light
from astropy.constants import h, k_B

C_LIGHT_KM_S = c_light.value / 1e3
H_OVER_K_B = h.value / k_B.value

def planck(nu_hz, t_k):
    """Return occupation number at frequency nu_hz and temperature t_k"""
    return 1 / (np.exp(H_OVER_K_B * nu_hz / t_k) - 1)

def compute_scalar_product(theta, phi, v):
    """Return the scalar (dot) product between a given direction and a velocity"""
    dx, dy, dz = np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)

    return dx * v[0] + dy * v[1] + dz * v[2]

def calculate_beta(theta, phi, v_km_s):
    """Return a 2-tuple containing β·n and β"""
    beta_dot_n = compute_scalar_product(theta, phi, v_km_s) / C_LIGHT_KM_S
    beta = np.sqrt(v_km_s[0] ** 2 + v_km_s[1] ** 2 + v_km_s[2] ** 2) / C_LIGHT_KM_S

    return beta_dot_n, beta

# Full formula in linearized units (the most widely used):
def compute_dipole_for_one_sample_total_from_lin_t(
    theta, phi, v_km_s, t_cmb_k, nu_hz, f_x, planck_t0
):
    beta_dot_n, beta = calculate_beta(theta, phi, v_km_s)
    gamma = 1 / np.sqrt(1 - beta ** 2)

    planck_t = planck(nu_hz, t_cmb_k / gamma / (1 - beta_dot_n))

    return t_cmb_k / f_x * (planck_t / planck_t0 - 1)

In [60]:
t_cmb_k = 2.725
nu_hz = 140*1e9
f_x = 1
planck_t0 = 1

theta = np.ones(len(vel_test[0]))
phi = np.ones(len(vel_test[0]))

compute_scalar_product(theta,phi,vel_test)

print(compute_dipole_for_one_sample_total_from_lin_t(theta,phi,vel_test,t_cmb_k,nu_hz,f_x,planck_t0))

[-2.47348424 -2.47172687 -2.47229014]
